In [ ]:
!pip install transformers
!pip install peft
!pip install 'accelerate>=0.26.0'
!pip install -U bitsandbytes
!pip install huggingface-hub
!pip install datasets
!pip install wandb

In [1]:
import json
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          Trainer,
                          pipeline,
                          DataCollatorForLanguageModeling,
                          PreTrainedTokenizer)
from peft import LoraConfig, get_peft_model
import huggingface_hub
import os
import logging
from tqdm import tqdm
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from typing import List
import wandb
from lora_llm import finetuning, evaluate, inference

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
os.environ['WANDB_NOTEBOOK_NAME'] = 'lora_llm'
os.environ["WANDB_DISABLED"] = "false"

# Load the API key from the secret.json file
with open('secrets.json', 'r') as file:
    secrets = json.load(file)
    huggingface_hub.login(secrets.get('HF_KEY'))
    wandb.login(key=secrets.get('WANDB_KEY'))

lr = 2e-4
epochs = 7
class_names = []
base_model_name = 'mistralai/Ministral-8B-Instruct-2410'
label_name = "evasion_label"
fine_tuned_model_path = f"./ministral_{epochs}ep"

# Wandb configuration
run = wandb.init(entity="kontilenia-national-technical-university-of-athens",
                 project='political-speech-clarity',
                 job_type="training",
                 name=str(epochs)+" epoch Ministral",
                 # Track hyperparameters and run metadata
                 config={
                    "learning_rate": lr,
                    "architecture": base_model_name,
                    "dataset": "qevasion_dataset_preproccessed",
                    "epochs": epochs,
                 })

model, tokenizer = finetuning(base_model_name,
                              fine_tuned_model_path,
                              label_name,
                              lr,
                              epochs)

evaluate(base_model_name,
         fine_tuned_model_path,
         "evasion_label",
         "clarity_label",
         "preprocessed_data/test_set_unnamed.csv",
         False,
         model,
         tokenizer,
         run)

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find lora_llm.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/ec2-user/.netrc
wandb: Currently logged in as: kontilenia (kontilenia-national-technical-university-of-athens) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

trainable params: 7667712 || all params: 4554592256
        || trainable%: 0.16835122814559145
Found 3109 instances for training and
    339 instances for validation.
Training...


Step,Training Loss,Validation Loss


KeyboardInterrupt: 